In [88]:
import openpyxl
import polars as pl

In [89]:
ruta = "data/opiniones.xlsx"
workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames


In [90]:

def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index].lower().replace(" ", "_")
    print(f"Nombre de la hoja: {nombre_hoja}")
    return nombre_hoja


def calificaciones(calificacion:str) -> int:
    """Convierte la calificación de string a int."""
    calificacion_map = {
        "Pésimo": 1,
        "Malo": 2,
        "Regular": 3,
        "Muy bueno": 4,
        "Excelente": 5
    }
    return calificacion_map.get(calificacion, 0)


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col


def añadir_columna_lugar_turistico(iteracion: int) -> pl.LazyFrame:
    df_excel = pl.read_excel(ruta, sheet_id=iteracion+1).lazy()
    nombre_hoja = obtener_nombre_hoja(iteracion)
    
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico"),
            pl.col("Calificación").map_elements(calificaciones).alias("calificacion_numerica")
        ])
    )

    df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.collect_schema().names()})
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel




In [91]:
lista_df: list[pl.LazyFrame] = []
for i, hoja in enumerate(hojas):
    lista_df.append(añadir_columna_lugar_turistico(i))

df_excel = juntar_df(lista_df)


Nombre de la hoja: basilica_colegiata_560
Nombre de la hoja: mercado_hidalgo_600
Nombre de la hoja: casa_de_diego_rivera_698
Nombre de la hoja: universidad_de_guanajuato_900
Nombre de la hoja: alhóndiga_930
Nombre de la hoja: teatro_juárez_1,010
Nombre de la hoja: jardín_de_la_unión_1,134
Nombre de la hoja: callejón_del_beso_1,360
Nombre de la hoja: monumento_pípila_1,620
Nombre de la hoja: museo_de_las_momias_1,650


In [92]:
# Guardar el DataFrame en un parquet
ruta_parquet = "data/opiniones_lugar_turistico.parquet"

df_excel = df_excel.collect()
df_excel.write_parquet(ruta_parquet)
print(f"Archivo guardado en: {ruta_parquet}")

Archivo guardado en: data/opiniones_lugar_turistico.parquet


In [93]:

def estadisticos(lf: pl.LazyFrame) -> pl.DataFrame:
    stats = (
        lf
        .group_by("lugar_turistico")
        .agg([
            # promedio y desviación estándar de la calificación
            pl.col("calificacion_numerica").mean().alias("promedio_calificacion"),
            pl.col("calificacion_numerica").std().alias("desviacion_estandar_calificacion"),

            # longitud de la opinión en número de palabras (promedio y std)
            pl.col("opinión")
              .str.split(" ")
              .list.len()
              .mean()
              .alias("longitud_promedio_opiniones"),

            pl.col("opinión")
              .str.split(" ")
              .list.len()
              .std()
              .alias("desviacion_estandar_longitud_opiniones"),

            # edad promedio y std nada más, para tener idea básica de (c)
            pl.col("edad").mean().alias("edad_promedio"),
            pl.col("edad").std().alias("edad_desviacion_estandar"),

            # % nacional como aproximación a (d)
            (
                (pl.col("nacional_ó_internacional") == "nacional")
                .cast(pl.Int8)
                .mean()
                .alias("porcentaje_nacional")
            ),
        ])
    )

    return stats.collect()



estadisticos(df_reseñas)

lugar_turistico,promedio_calificacion,desviacion_estandar_calificacion,longitud_promedio_opiniones,desviacion_estandar_longitud_opiniones,edad_promedio,edad_desviacion_estandar,porcentaje_nacional
str,f64,f64,f64,f64,f64,f64,f64
"""universidad_de_guanajuato_900""",4.603333,0.612613,35.391111,20.035073,39.311673,13.929438,0.0
"""basilica_colegiata_560""",4.498214,0.668748,39.716071,29.414071,41.874263,14.836906,0.0
"""museo_de_las_momias_1,650""",3.675152,1.136495,58.154545,46.259502,38.964744,12.886166,0.0
"""jardín_de_la_unión_1,134""",4.593474,0.602724,43.118166,36.804919,41.385827,14.89092,0.0
"""teatro_juárez_1,010""",4.70297,0.518146,41.511881,27.091014,41.75383,13.813232,0.0
"""monumento_pípila_1,620""",4.377778,0.76795,47.35,35.763244,38.436513,13.04939,0.0
"""callejón_del_beso_1,360""",3.995588,0.971247,45.522059,32.872547,38.82622,13.335956,0.0
"""casa_de_diego_rivera_698""",4.24212,0.805364,49.280802,30.468154,44.348092,15.124692,0.0
"""alhóndiga_930""",4.454839,0.745711,46.504301,33.822568,40.706593,13.343588,0.0
